In [2]:
!pip install yellowbrick

In [3]:
import pandas as pd

import nltk
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer

In [4]:
#Считываем стоп-слова
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') 
stop_words = set(stopwords.words('russian')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\14675\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#Читаем лемматизированные записи из множества файлов
count = 1000
max_count = 824000
df_l = pd.DataFrame()
while count <= max_count:
    df_1000 = pd.read_csv(r"lemmatized"+str(count)+".csv", 
                 on_bad_lines='warn', 
                 delimiter=';',
                 na_values='',
                 dtype='str'
                )
    df_l = pd.concat([df_l, df_1000], ignore_index=True)
    count += 1000
df_l

,Unnamed: 0,id,name,ntd,ntd_m,ens,ens_id,source,lemmatized_name
0,1000,1000193915,Круг h11-МД-25 ГОСТ 7417-75 / 10Х11Н23Т3МР (ЭИ...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-МД-25 гост 7417-75 / 10Х11Н23Т3МР (ЭИ...
1,1001,1000193940,Круг h11-НД-6 ГОСТ 7417-75 / 12Х18Н9Т-Б ТУ 14-...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-НД-6 гост 7417-75 / 12Х18Н9Т-б тот 14...
2,1002,1000193920,Круг h11-МД-10 ГОСТ 7417-75 / 20Х13-В ТУ 14-1-...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-МД-10 гост 7417-75 / 20Х13-в тот 14-1...
3,1003,1000193934,Круг h11-НД-12 ГОСТ 7417-75 / 12Х18Н9Т-Б ТУ 14...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-НД-12 гост 7417-75 / 12Х18Н9Т-б тот 1...
4,1004,1000193912,Круг h11-МД-17 ГОСТ 7417-75 / 10Х11Н23Т3МР (ЭИ...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-МД-17 гост 7417-75 / 10Х11Н23Т3МР (ЭИ...
...,...,...,...,...,...,...,...,...,...
823739,824739,7914370148.000,"УПЛОТНИТЕЛЬ,",NaN,NaN,NaN,NaN,candidate,"уплотнитель,\n"
823740,824740,9990297149.000,"ФИЛЬТР ВОЗДУШНЫЙ,",NaN,NaN,NaN,NaN,candidate,"фильтр воздушный,\n"
823741,824741,9990297149.200,"Фильтр салонный,",NaN,NaN,NaN,NaN,candidate,"фильтр салонный,\n"
823742,824742,9990297210.000,"ФИЛЬТР ТОПЛИВНЫЙ,",NaN,NaN,NaN,NaN,candidate,"фильтр топливный,\n"


In [6]:
corpus = list(df_l['lemmatized_name'])
stop_words = list(stopwords.words('russian'))

"""
# Cоздаём N-граммы из одного слова слов
count_vect = CountVectorizer()
bow1 = count_vect.fit_transform(corpus)
print("Размер мешка без учёта стоп-слов:", bow1.shape)

#То же самое, но исключаем стоп слова
count_vect = CountVectorizer(stop_words=stop_words)
bow2 = count_vect.fit_transform(corpus)
print("Размер мешка с учётом стоп-слов:", bow2.shape)
"""

#N-граммы по 2 слова
count_vect = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words)
bow3 = count_vect.fit_transform(corpus)
print("Размер мешка ду-грамм с учётом стоп-слов:", bow3.shape)

#Готовим датафрейм для визуализации кластеров при помощий PCA и визуализации более лаконичным способом
df_bow = pd.DataFrame.sparse.from_spmatrix(bow3)

Размер мешка ду-грамм с учётом стоп-слов: (823744, 515031)


#Визуализация графика для метода локтя ещё одним способом
distortion = []

K = range(1, 120)
for k in K:
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(bow3)
    distortion.append(model.inertia_)

plt.figure(figsize=(12, 8))
plt.plot(K, distortion, 'bx-')
plt.xlabel('Число кластеров')
plt.ylabel('Значение целевой функции')
plt.show()

print("Целевая функция (конечное значение):")
print(model.inertia_)

#Add cluster labels to the DataFrame
df_l['cluster'] = model.labels_

#Визуализация графика для метода локтя ещё одним более лаконичным способом
# Instantiate the clustering model and visualizer
km = KMeans(random_state=42)
visualizer = KElbowVisualizer(km, k=(2,120))
 
visualizer.fit(df_bow)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

#Полезная визуализация, когда есть относительно небольшое количество кластеров, но это не наш случай. Поэтому отключаем

fig, ax = plt.subplots(14, 3, figsize=(18,60))
for i in range (3, 121, 3):
    '''
    Create KMeans instances for different number of clusters
    '''
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
    q, mod = divmod(int(i/3+2), 3)
    '''
    Create SilhouetteVisualizer instance with KMeans instance
    Fit the visualizer
    '''
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(df_bow) 

### Metics

In [11]:
from sklearn.metrics import silhouette_score, davies_bouldin_score,v_measure_score

### Running k-means and computing inter-cluster distance score for various k values

In [ ]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
for i in range(2,2022, 20):
    km = KMeans(n_clusters=i, random_state=0).fit(bow3)
    preds = km.predict(bow3)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(bow3)))
    km_scores.append(-km.score(bow3))
    
    silhouette = silhouette_score(bow3,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
       

Score for number of cluster(s) 2: -5817107.317239519
Silhouette score for number of cluster(s) 2: -0.02366773131341135
Score for number of cluster(s) 22: -4988380.842396112
Silhouette score for number of cluster(s) 22: -0.08227343312272821
Score for number of cluster(s) 42: -4726042.226796935
Silhouette score for number of cluster(s) 42: -0.10529278065852073
Score for number of cluster(s) 62: -4537989.348090344
Silhouette score for number of cluster(s) 62: -0.08525155272989088
Score for number of cluster(s) 82: -4415882.885011139
Silhouette score for number of cluster(s) 82: -0.07437002525550473
Score for number of cluster(s) 102: -4328834.91495378
Silhouette score for number of cluster(s) 102: -0.06982111747446074
Score for number of cluster(s) 122: -4267321.211983875
Silhouette score for number of cluster(s) 122: -0.0625503257826761
Score for number of cluster(s) 142: -4211892.177485001
Silhouette score for number of cluster(s) 142: -0.05997837522919269
Score for number of cluster(s)

In [ ]:
plt.figure(figsize=(18,10))
plt.title("The elbow method for determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,2022,20)],y=km_scores,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("K-means score",fontsize=14)
plt.xticks([i for i in range(2,2022,20)],fontsize=6)
plt.yticks(fontsize=6)
plt.show()

In [ ]:
plt.figure(figsize=(18,10))
plt.title("The silhouette coefficient method \nfor determining number of clusters\n",fontsize=16)
plt.scatter(x=[i for i in range(2,2022,20)],y=km_silhouette,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Number of clusters",fontsize=14)
plt.ylabel("Silhouette score",fontsize=14)
plt.xticks([i for i in range(2,2022,20)],fontsize=6)
plt.yticks(fontsize=6)
plt.show()

График метрики, вычисленной по методу Силуэт имеет чётко выраженный пик на 842 кластерах. Поэтому финальную кластеризацию проведём на этом количестве. 

In [19]:
#Окончательная кластеризация с заданным количеством кластеров

model = KMeans(n_clusters=1082, random_state=42)
model.fit(bow3)
df_l['cluster'] = model.labels_

# Display the DataFrame with cluster labels
display(df_l)

#Оставляем только те кластера, где есть и master и candidate
# Step 1: Identify clusters with more than one unique source
clusters_with_multiple_sources = df_l.groupby('cluster')['source'].nunique()
clusters_to_keep = clusters_with_multiple_sources[clusters_with_multiple_sources > 1].index

# Step 2: Filter the DataFrame
filtered_df = df_l[df_l['cluster'].isin(clusters_to_keep)]

# Display the filtered DataFrame
#display(filtered_df)
filtered_df.to_csv('filtered_1082_siluette.csv', encoding='utf-8-sig', sep=';')

,Unnamed: 0,id,name,ntd,ntd_m,ens,ens_id,source,lemmatized_name,cluster
0,1000,1000193915,Круг h11-МД-25 ГОСТ 7417-75 / 10Х11Н23Т3МР (ЭИ...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-МД-25 гост 7417-75 / 10Х11Н23Т3МР (ЭИ...,21
1,1001,1000193940,Круг h11-НД-6 ГОСТ 7417-75 / 12Х18Н9Т-Б ТУ 14-...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-НД-6 гост 7417-75 / 12Х18Н9Т-б тот 14...,1027
2,1002,1000193920,Круг h11-МД-10 ГОСТ 7417-75 / 20Х13-В ТУ 14-1-...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-МД-10 гост 7417-75 / 20Х13-в тот 14-1...,794
3,1003,1000193934,Круг h11-НД-12 ГОСТ 7417-75 / 12Х18Н9Т-Б ТУ 14...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-НД-12 гост 7417-75 / 12Х18Н9Т-б тот 1...,1027
4,1004,1000193912,Круг h11-МД-17 ГОСТ 7417-75 / 10Х11Н23Т3МР (ЭИ...,ГОСТ 7417-75,ТУ 14-1-3957-85,"Калиброванные круги, шестигранники, квадраты",01.01.02.02,master,круг h11-МД-17 гост 7417-75 / 10Х11Н23Т3МР (ЭИ...,21
...,...,...,...,...,...,...,...,...,...,...
823739,824739,7914370148.000,"УПЛОТНИТЕЛЬ,",NaN,NaN,NaN,NaN,candidate,"уплотнитель,\n",132
823740,824740,9990297149.000,"ФИЛЬТР ВОЗДУШНЫЙ,",NaN,NaN,NaN,NaN,candidate,"фильтр воздушный,\n",132
823741,824741,9990297149.200,"Фильтр салонный,",NaN,NaN,NaN,NaN,candidate,"фильтр салонный,\n",132
823742,824742,9990297210.000,"ФИЛЬТР ТОПЛИВНЫЙ,",NaN,NaN,NaN,NaN,candidate,"фильтр топливный,\n",132


In [ ]:
#Дополняем датасет колонкой cluster
df_bow['cluster'] = model.labels_
df_bow.columns = df_bow.columns.astype(str)
print(df_bow.columns)

In [ ]:
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_bow)
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_data)
# Create a DataFrame with the principal components and cluster labels
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['cluster'] = df_bow['cluster'].values
plt.figure(figsize=(10, 6))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['cluster'], cmap='viridis', alpha=0.6, edgecolors='w', s=100)

# Create a legend
legend1 = plt.legend(*scatter.legend_elements(), title="Clusters")
plt.gca().add_artist(legend1)

plt.title('PCA Visualization of Clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid()
plt.show()